<a href="https://colab.research.google.com/github/Shubham619/Transfer-learning_Donorschoose-screening/blob/master/Transfer_learning_Donorschoose.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import random

In [ ]:

from fastai.text import *

In [ ]:
train=train.loc[:,['project_is_approved','text']]

In [ ]:
train.columns=['label','text']

In [ ]:
labels={1:"accepted",0:"rejected"}
train['label']=train['label'].map(labels)

In [ ]:

rand=random.sample(range(0, len(train)), 120000)

In [ ]:
train=train.loc[rand,:]

In [ ]:
train=train.reset_index()
train=train.loc[:,["label","text"]]

In [ ]:
train.to_csv('texts.csv')

In [ ]:
import pandas as pd
train=pd.read_csv("/content/drive/My Drive/DonorsChoose/texts.csv")

In [ ]:
df_trn,df_val=train_test_split(train,stratify=train['label'],test_size=0.3,random_state=12)

In [ ]:
print(df_trn.columns)
print(train.columns)

Index(['label', 'text'], dtype='object')
Index(['label', 'text'], dtype='object')


In [ ]:
data_lm=TextLMDataBunch.from_df(train_df=df_trn,valid_df=test,path="")

In [ ]:
data_cls=TextClasDataBunch.from_df(path="",train_df=df_trn,valid_df=test,vocab=data_lm.train_ds.vocab,bs=32)

In [ ]:

data_lm.save('data_lm_export.pkl')
data_cls.save('data_cls_export.pkl')

In [ ]:
bs=32

data_lm = load_data(path, 'data_lm_export.pkl', bs=bs)
data_cls = load_data(path, 'data_cls_export.pkl', bs=bs)

In [ ]:
torch.cuda.set_device(0)

In [ ]:

learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5)

In [ ]:

learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,3.417661,3.188269,0.367963,1:08:32


In [ ]:
learn.unfreeze()
learn.fit_one_cycle(3, slice(1e-4,1e-2))

epoch,train_loss,valid_loss,accuracy,time
0,3.125122,2.961061,0.398273,1:18:05
1,2.915049,2.809442,0.417198,1:18:02


Buffered data was truncated after reaching the output size limit.

In [ ]:
learn.save('mini_donorchoose_language_model')
learn.save_encoder('mini_donorchoose_language_model_encoder')

In [ ]:
path="/content/drive/My Drive/DonorsChoose"

In [ ]:
learn = text_classifier_learner(data_cls, AWD_LSTM, drop_mult=0.5).to_fp16()
learn.load('/content/drive/My Drive/DonorsChoose/export')

RNNLearner(data=TextClasDataBunch;

Train: LabelList (84000 items)
x: TextList
xxbos math & science , applied learning applied sciences , other ipad for istudents ! my students are starting off their academic careers at turner / bartels ! turner / bartels is a stem school that leads many opportunities to use technology to enhance the learning of the child . my students have the opportunity to reach past all boundaries with technology and different learning ! my students are from all different cultural backgrounds and have a multitude of experiences . a majority of my students come from he new tampa area and have walked many paths . please help them reach the stars ! thank you ! having technology at the finger tips of each of my students allows them to understand and investigate more than what is brought into the classroom . my students will be researching cultures , animals , and a multitude of different things . technology enhances children 's learning by giving them boundless opportu

In [ ]:
data_cls.show_batch()

text,target
"xxbos math & science , music & the arts environmental science , visual arts hey ! what 's cooking ? a clay program for art classes ! the students at park creek school are 90 % english is their second language . they come from cultures all over the world . pakistan , japan , vietnam , india , honduras , nigeria , venezuela and mexico are just some of",accepted
"xxbos math & science environmental science sustainable green classroom i teach environmental science to a multi - level high school students composed of 116 all - boys 9th and 10th graders . each session encompasses 14 - 19 students in an inclusive classroom where student motivation is primarily derived from hands - on , inquiry - based , and project - supported learning . our most recent lesson was about",accepted
"xxbos history & civics civics & government , history & geography west virginia studies laptops providing for the future ! tucked away in coal , oil , and natural gas country in rural west virginia , my students are challenged economically . living in rural west virginia challenges my students to be successful in our changing world . our students learn through technology and we need to adapt to their",accepted
"xxbos health & sports health & wellness if you can dream it , you can achieve it ! all of my students are diagnosed with emotional , behavioral , and social disabilities . they struggle to control their feelings and behavior in an effort to maintain a calm demeanor while they try to navigate through their school day . often , they are easily frustrated during routine academic demands ,",accepted
"xxbos health & sports health & wellness green thumbs creating healthy bodies although the majority of my students come from poverty - stricken homes , they are some of the hardest working , intellectual students you will meet . my students are faced with challenges every day , but they do not allow such challenges to defeat their purposes in life . all students in my school receive free lunch",accepted


In [ ]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,0.381988,0.375582,0.849861,05:39


In [ ]:

learn.unfreeze()
learn.fit_one_cycle(5, slice(1e-4, 1e-2))

epoch,train_loss,valid_loss,accuracy,time
0,0.360455,0.378693,0.856889,11:25
1,0.359361,0.370168,0.848889,11:22
2,0.327215,0.387760,0.860417,11:21
3,0.322132,0.358733,0.860556,11:24
4,0.306868,0.390769,0.860444,11:25


In [ ]:

learn.unfreeze()
learn.fit_one_cycle(1, slice(1e-4, 1e-2))

In [ ]:

learn.unfreeze()
learn.fit_one_cycle(2, slice(1e-4, 1e-2))

epoch,train_loss,valid_loss,accuracy,time
0,0.350615,0.369051,0.857861,11:07
1,0.340533,0.349145,0.861194,11:08


In [ ]:
preds, targets= learn.get_preds()

predictions = np.argmax(preds, axis = 1)
pd.crosstab(predictions, target)

col_0,0,1
row_0,,
0,29802,4315
1,709,1174


In [ ]:
(29802+1174)/(29802+1174+4315+709)

0.8604444444444445

In [ ]:
learn.save('mini_donorchoose_language_model_final')